In [10]:
#import packages
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, JoinNode,MapNode      # pypeline engine

from nipype.interfaces.fsl.model import Randomise, GLM, Cluster
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.fsl.utils import ImageMeants, Merge, Split
from nipype.interfaces.fsl.maths import ApplyMask
from pandas import read_csv, DataFrame

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
setup = "Lucinda"

if setup == "sherlock":
    studyhome = '/oak/stanford/groups/iang/BABIES_data/BABIES_rest/'
    preproc_dir = studyhome + 'processed/preproc'
    output_dir = studyhome + 'processed/sbc_analysis'
    workflow_dir = studyhome + 'workflows'
    roi_dir = studyhome + 'ROIs'
    group_con = studyhome + 'misc/tcon.con'
    template_brain = studyhome + 'templates/T2w_BABIES_template_2mm.nii.gz'
    gm_template = studyhome + 'templates/BABIES_gm_mask_2mm.nii.gz'

elif setup == "Lucinda":
    studyhome = '/Volumes/iang/BABIES_data/BABIES_rest/'
    preproc_dir = studyhome + 'processed/preproc'
    #output_dir = studyhome + 'processed/sbc_analysis
    output_dir = studyhome + 'processed/sbc_analysis_noAge' #New datasink directory
    workflow_dir = studyhome + 'workflows'
    roi_dir = studyhome + 'ROIs'
    group_con = studyhome + 'misc/tcon_noAge.con' #Contrast file without column for age
    template_brain = studyhome + 'templates/T2w_BABIES_template_2mm.nii.gz'
    gm_template = studyhome + 'templates/BABIES_gm_mask_2mm.nii.gz'
    subject_info = read_csv(studyhome + 'misc/subject_info_srcd19.csv', index_col=None)
    

elif setup == "Lucy":
    studyhome = '/Volumes/iang/BABIES_data/BABIES_rest/'
    preproc_dir = studyhome + 'processed/preproc'
    output_dir = studyhome + 'processed/sbc_analysis'
    workflow_dir = studyhome + 'workflows'
    roi_dir = studyhome + 'ROIs'
    group_con = studyhome + 'misc/tcon.con'
    template_brain = studyhome + 'templates/T2w_BABIES_template_2mm.nii.gz'
    gm_template = studyhome + 'templates/BABIES_gm_mask_2mm.nii.gz'
    

elif setup == "Cat":
    studyhome = '/Users/catcamacho/Box/SNAP/BABIES/BABIES_rest/'
    preproc_dir = studyhome + 'processed/preproc'
    output_dir = studyhome + 'processed/sbc_analysis'
    workflow_dir = studyhome + 'workflows'
    roi_dir = studyhome + 'ROIs'
    group_con = studyhome + 'misc/tcon.con'
    template_brain = studyhome + 'templates/T2w_BABIES_template_2mm.nii.gz'
    gm_template = studyhome + 'templates/BABIES_gm_mask_2mm.nii.gz'

#subject_info = read_csv(studyhome + 'misc/subject_info.csv', index_col=None) #csv with columns 'subject_id','age', and any group lists like 'lena'
#### REMOVED AGE AND SEX FOR SRCD ANALYSES ###
subject_info = subject_info.sort_values(by=['subject_id']).drop(['age', 'sex'], axis=1)
subjects_list = subject_info['subject_id'].tolist()
list_names = ['lena','sens', 'postnatal_stress'] #replace with actual group list labels

proc_cores = 1

# ROIs for connectivity analysis
Lamyg = roi_dir + '/L_amyg_6mm.nii.gz'
Ramyg = roi_dir + '/R_amyg_6mm.nii.gz'
lipl = roi_dir + '/l_ipl_8mm.nii.gz'
ripl = roi_dir + '/r_ipl_8mm.nii.gz'
mpfc = roi_dir + '/mPFC_8mm_split.nii.gz'

# ROIs = [Lamyg, Ramyg]
# rois = ['L_amyg','R_amyg']

Lifg = roi_dir + '/l_ifg_10mm.nii.gz'
Rifg = roi_dir + '/r_ifg_10mm.nii.gz'

ROIs = [Lifg, Rifg]
rois = ['l_ifg','r_ifg']

target_rois = [Lifg, Rifg, Lamyg, Ramyg]
min_clust_size = 10

In [11]:
## File handling
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id','ROIs']),
                     name='infosource')
infosource.iterables = [('subject_id', subjects_list),('ROIs',ROIs)]


# Data grabber- select fMRI and ROIs
templates = {'func': preproc_dir + '/proc_func/{subject_id}/func_filtered_smooth.nii*'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
substitutions = [('_subject_id_', ''),
                ('_ROIs_..Users..catcamacho..Box..SNAP..BABIES..BABIES_rest..ROIs..',''), 
                 ('_ROIs_..share..iang..active..BABIES..BABIES_rest..ROIs..',''), 
                  ('_ROIs_..oak..stanford..groups..iang..BABIES_data..BABIES_rest..ROIs..',''),
                 ('_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..',''),
                 ('_6mm.nii.gz','_'),('_8mm.nii.gz','_'),('_10mm.nii.gz','_')]
datasink.inputs.substitutions = substitutions

In [12]:
## Seed-based level 1

# Extract ROI timeseries
ROI_timeseries = Node(ImageMeants(), name='ROI_timeseries', iterfield='mask')

# model ROI connectivity
glm = Node(GLM(out_file='betas.nii.gz',out_cope='cope.nii.gz'), name='glm', iterfield='design')

In [13]:
sbc_workflow = Workflow(name='sbc_workflow')
sbc_workflow.connect([(infosource,selectfiles,[('subject_id','subject_id')]),
                      (selectfiles,ROI_timeseries,[('func','in_file')]),
                      (infosource,ROI_timeseries,[('ROIs','mask')]),
                      (ROI_timeseries,glm,[('out_file','design')]),
                      (selectfiles,glm,[('func','in_file')]),
                      (glm,datasink,[('out_file','glm_betas')])
                     ])
sbc_workflow.base_dir = workflow_dir
#sbc_workflow.write_graph(graph2use='flat')
sbc_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})

190313-13:59:37,24 nipype.workflow INFO:
	 Workflow sbc_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190313-13:59:41,392 nipype.workflow INFO:
	 Running in parallel.
190313-13:59:41,401 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 70 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 1/1.
190313-13:59:42,346 nipype.workflow INFO:
	 [Node] Setting-up "sbc_workflow.selectfiles" in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/sbc_workflow/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..r_ifg_10mm.nii.gz_subject_id_143-C-T1/selectfiles".
190313-13:59:43,404 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 69 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 0/1.
                     Currently running:
                       * sbc_workflow.selectfiles
190313-13:59:44,61 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
190313-13:59:46,313 nipype.workflow INFO:
	 [Node] Finished "s

190313-14:00:35,802 nipype.workflow INFO:
	 [Job 13] Cached (sbc_workflow.ROI_timeseries).
190313-14:00:37,696 nipype.workflow INFO:
	 [Job 14] Cached (sbc_workflow.glm).
190313-14:00:39,930 nipype.workflow INFO:
	 [Node] Setting-up "sbc_workflow.datasink" in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/sbc_workflow/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..l_ifg_10mm.nii.gz_subject_id_140-C-T1/datasink".
190313-14:00:41,219 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
190313-14:00:41,265 nipype.interface INFO:
	 sub: /Volumes/iang/BABIES_data/BABIES_rest/processed/sbc_analysis_noAge/glm_betas/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..l_ifg_10mm.nii.gz_subject_id_140-C-T1/betas.nii.gz -> /Volumes/iang/BABIES_data/BABIES_rest/processed/sbc_analysis_noAge/glm_betas/l_ifg_140-C-T1/betas.nii.gz
190313-14:00:41,464 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 66 jobs ready. Free memory (GB): 28.60/28.80, Free p

190313-14:01:31,741 nipype.workflow INFO:
	 [Node] Setting-up "sbc_workflow.selectfiles" in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/sbc_workflow/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..l_ifg_10mm.nii.gz_subject_id_132-C-T1/selectfiles".
190313-14:01:33,514 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 62 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 0/1.
                     Currently running:
                       * sbc_workflow.selectfiles
190313-14:01:34,23 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
190313-14:01:34,477 nipype.workflow INFO:
	 [Node] Finished "sbc_workflow.selectfiles".
190313-14:01:35,513 nipype.workflow INFO:
	 [Job 28] Completed (sbc_workflow.selectfiles).
190313-14:01:35,517 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 63 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 1/1.
190313-14:01:35,817 nipype.workflow INFO:
	 [Job 29] Cached (sbc_workfl

	 [Node] Setting-up "sbc_workflow.datasink" in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/sbc_workflow/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..r_ifg_10mm.nii.gz_subject_id_129-C-T1/datasink".
190313-14:02:19,554 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 59 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 0/1.
                     Currently running:
                       * sbc_workflow.datasink
190313-14:02:28,947 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
190313-14:02:29,2 nipype.interface INFO:
	 sub: /Volumes/iang/BABIES_data/BABIES_rest/processed/sbc_analysis_noAge/glm_betas/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..r_ifg_10mm.nii.gz_subject_id_129-C-T1/betas.nii.gz -> /Volumes/iang/BABIES_data/BABIES_rest/processed/sbc_analysis_noAge/glm_betas/r_ifg_129-C-T1/betas.nii.gz
190313-14:02:31,176 nipype.workflow INFO:
	 [Node] Finished "sbc_workflow.datasink".
190313-14:02:31,560 nipype.

190313-14:03:13,803 nipype.workflow INFO:
	 [Node] Setting-up "sbc_workflow.selectfiles" in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/sbc_workflow/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..r_ifg_10mm.nii.gz_subject_id_124-C-T1/selectfiles".
190313-14:03:15,601 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 55 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 0/1.
                     Currently running:
                       * sbc_workflow.selectfiles
190313-14:03:18,520 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
190313-14:03:20,941 nipype.workflow INFO:
	 [Node] Finished "sbc_workflow.selectfiles".
190313-14:03:21,605 nipype.workflow INFO:
	 [Job 56] Completed (sbc_workflow.selectfiles).
190313-14:03:21,608 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 56 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 1/1.
190313-14:03:21,939 nipype.workflow INFO:
	 [Job 57] Cached (sbc_workf

	 [Node] Setting-up "sbc_workflow.datasink" in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/sbc_workflow/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..l_ifg_10mm.nii.gz_subject_id_122-C-T1/datasink".
190313-14:04:08,846 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
190313-14:04:08,893 nipype.interface INFO:
	 sub: /Volumes/iang/BABIES_data/BABIES_rest/processed/sbc_analysis_noAge/glm_betas/_ROIs_..Volumes..iang..BABIES_data..BABIES_rest..ROIs..l_ifg_10mm.nii.gz_subject_id_122-C-T1/betas.nii.gz -> /Volumes/iang/BABIES_data/BABIES_rest/processed/sbc_analysis_noAge/glm_betas/l_ifg_122-C-T1/betas.nii.gz
190313-14:04:09,649 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 52 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 0/1.
                     Currently running:
                       * sbc_workflow.datasink
190313-14:04:11,81 nipype.workflow INFO:
	 [Node] Finished "sbc_workflow.datasink".
190313-14:04:11,649 nipype

Process Process-4:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [15]:
def extract_cluster_betas(cluster_index_file, sample_betas, min_clust_size, subject_ids):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, zeros_like
    from nipype.interfaces.fsl.utils import ImageMeants
    from os.path import abspath
    
    subject_ids = sorted(subject_ids)
    sample_data = DataFrame(subject_ids, index=None, columns=['Subject'])
    
    cluster_nifti = load(cluster_index_file[0])
    cluster_data = cluster_nifti.get_data()
    clusters, cluster_sizes = unique(cluster_data, return_counts=True)
    
    final_clusters = clusters[cluster_sizes>=min_clust_size]
    for clust_num in final_clusters[1:]:
        temp = zeros_like(cluster_data)
        temp[cluster_data==clust_num] = 1
        temp_nii = Nifti1Image(temp,cluster_nifti.affine)
        temp_file = 'temp_clust_mask.nii.gz'
        save(temp_nii, temp_file)
        
        eb = ImageMeants()
        eb.inputs.in_file = sample_betas
        eb.inputs.mask = temp_file
        eb.inputs.out_file = 'betas.txt'
        eb.run()
        betas = open('betas.txt').read().splitlines()
        sample_data['clust' + str(clust_num)] = Series(betas, index=sample_data.index)
    
    sample_data.to_csv('extracted_betas.csv')
    extracted_betas_csv = abspath('extracted_betas.csv')
    return(extracted_betas_csv)

def separate_sub_lists(subjects_df, list_name, output_dir, roi):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from subprocess import check_call
    from os.path import abspath
    from nipype.interfaces.fsl import Merge

    included_subs = subjects_df[subjects_df[list_name]==1]
    included_subs = included_subs.sort_values(by=['subject_id'])
    subjects_list = included_subs['subject_id'].tolist()
    #subjects_ages = included_subs['age'].tolist()

    beta_list = []
    text_file = open('temp_text.txt','w')
    for a in range(0,len(subjects_list)):
        beta_list.append(output_dir+'/glm_betas/'+roi+'_'+subjects_list[a]+'/betas.nii.gz')
        #text_file.write('1 {0}\n'.format(subjects_ages[a]))
        text_file.write('1\n')

    text_file.close()
    file = abspath('temp_text.txt')
    check_call(['Text2Vest',file,list_name + '_design.mat'])
    design_file = abspath('{0}_design.mat'.format(list_name))

    me=Merge()
    me.inputs.dimension='t'
    me.inputs.in_files=beta_list
    me.inputs.merged_file='betas_merged.nii.gz'
    me.run()

    betas = abspath('betas_merged.nii.gz')

    return(betas,design_file,subjects_list)

In [16]:
## Level 2
# new identity node
infosource2 = Node(IdentityInterface(fields=['roi','list_name']),
                   name='infosource2')
infosource2.iterables = [('roi',rois),('list_name',list_names)]

# Data grabber- select fMRI and ROIs
datagrabber = Node(Function(input_names=['subjects_df', 'list_name', 'output_dir', 'roi'], 
                            output_names=['betas','design_file', 'subjects_list'], 
                            function=separate_sub_lists), name='datagrabber')
datagrabber.inputs.output_dir = output_dir
datagrabber.inputs.subjects_df = subject_info

# FSL randomise for higher level analysis
highermodel = Node(Randomise(raw_stats_imgs= True,
                             mask=gm_template,
                             tcon=group_con),
                   name = 'highermodel')

## Cluster results
# make binary masks of sig clusters
binarize = MapNode(Binarize(min=0.95), 
                   name='binarize', 
                   iterfield=['in_file'])

# mask T-map before clustering
mask_tmaps = MapNode(ApplyMask(), 
                     name='mask_tmaps',
                     iterfield=['in_file','mask_file'])

# clusterize and extract cluster stats/peaks
clusterize = MapNode(Cluster(threshold=3.56, #t-stat for p=.001 (calculate based on N); default is FWE
                             out_index_file='outindex.nii.gz', 
                             out_localmax_txt_file='localmax.txt'), 
                     name='clusterize',
                     iterfield=['in_file'])

extract_betas = Node(Function(input_names=['cluster_index_file','sample_betas',
                                           'min_clust_size','subject_ids'],
                              output_names=['extracted_betas_csv'],
                              function=extract_cluster_betas),
                     name='extract_betas')
extract_betas.inputs.min_clust_size = min_clust_size

In [ ]:
group_workflow = Workflow(name='group_workflow')
group_workflow.connect([(infosource2,datagrabber,[('roi','roi')]),
                        (infosource2,datagrabber,[('list_name','list_name')]),
                        (datagrabber, highermodel,[('betas','in_file'),('design_file','design_mat')]),
                        (highermodel, clusterize, [('tstat_files','in_file')]),
                        (datagrabber, extract_betas, [('betas','sample_betas'),
                                                      ('subjects_list','subject_ids')]),
                        (clusterize, extract_betas, [('index_file','cluster_index_file')]),

                        (highermodel,datasink,[('t_corrected_p_files','rand_corrp_files')]),
                        (highermodel,datasink,[('tstat_files','rand_tstat_files')]),
                        (clusterize,datasink,[('index_file','cluster_index_file')]),
                        (clusterize,datasink,[('localmax_txt_file','localmax_txt_file')]),
                        (datagrabber, datasink, [('betas','merged_betas')]),
                        (extract_betas, datasink, [('extracted_betas_csv','all_cluster_betas')])
                       ])
group_workflow.base_dir = workflow_dir
#group_workflow.write_graph(graph2use='flat')
group_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})

190313-14:04:33,632 nipype.workflow INFO:
	 Workflow group_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190313-14:04:35,535 nipype.workflow INFO:
	 Running in parallel.
190313-14:04:35,539 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 6 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 1/1.
190313-14:04:35,770 nipype.workflow INFO:
	 [Node] Setting-up "group_workflow.datagrabber" in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_postnatal_stress_roi_r_ifg/datagrabber".
190313-14:04:37,544 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 5 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 0/1.
                     Currently running:
                       * group_workflow.datagrabber
190313-14:04:40,294 nipype.workflow INFO:
	 [Node] Running "datagrabber" ("nipype.interfaces.utility.wrappers.Function")
190313-14:04:45,789 nipype.interface DEBUG:
	 in_files_['/Volumes/iang/BABIES_data/BABIES_rest/pro

24   143-C-T1     1   0.0               1.0         NaN   M), ('needed_outputs', ['betas', 'design_file', 'subjects_list'])], 8473048f29dbf415f6ca9b9d709ea272, /Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_sens_roi_r_ifg/datagrabber/_0x8473048f29dbf415f6ca9b9d709ea272.json, []
190313-14:05:38,130 nipype.workflow DEBUG:
	 [Node] No hashfiles found in "/Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_sens_roi_r_ifg/datagrabber".
190313-14:05:38,134 nipype.utils DEBUG:
	 Removing contents of /Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_sens_roi_r_ifg/datagrabber
190313-14:05:38,711 nipype.workflow DEBUG:
	 Unable to write a particular type to the json file
190313-14:05:38,750 nipype.utils DEBUG:
	 Creating directory /Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_sens_roi_r_ifg/datagrabber/_report
190313-14:05:38,792 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Volum

            * tcon: ('/Volumes/iang/BABIES_data/BABIES_rest/misc/tcon_noAge.con', '1e975ee202a628790c89304f3574192c') != ['/Volumes/iang/BABIES_data/BABIES_rest/misc/tcon.con', '8af611acbbe514d74552ad7fe894afda']
190313-14:06:06,82 nipype.utils DEBUG:
	 Removing contents of /Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_sens_roi_r_ifg/highermodel
190313-14:06:06,806 nipype.utils DEBUG:
	 Creating directory /Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_sens_roi_r_ifg/highermodel/_report
190313-14:06:06,867 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Volumes/iang/BABIES_data/BABIES_rest/workflows/group_workflow/_list_name_sens_roi_r_ifg/highermodel/_report/report.rst"
190313-14:06:07,643 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 6 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 0/1.
                     Currently running:
                       * group_workflow.highermodel
190313-14:06:07,7

In [ ]:
def target_roi_betas(target_rois, sample_betas, subject_ids):
    from nipype.interfaces.fsl.utils import ImageMeants
    from os.path import abspath, basename
    from pandas import DataFrame, Series
    from nipype import config, logging

    config.enable_debug_mode()
    logging.update_logging(config)

    subject_ids = sorted(subject_ids)
    sample_data = DataFrame(subject_ids, index=None, columns=['Subject'])

    for roi in target_rois: 
        roi_file = basename(roi)
        eb = ImageMeants()
        eb.inputs.in_file = sample_betas
        eb.inputs.mask = roi
        eb.inputs.out_file = 'betas.txt'
        eb.run()
        betas = open('betas.txt').read().splitlines()
        sample_data[roi_file[:-7]] = Series(betas, index=sample_data.index)

    sample_data.to_csv('extracted_betas.csv')
    extracted_betas_csv = abspath('extracted_betas.csv')
    return(extracted_betas_csv)

target_betas = Node(Function(input_names=['target_rois','sample_betas','subject_ids'], 
                             output_names=['extracted_betas_csv'], 
                             function=target_roi_betas),
                    name='target_betas')
target_betas.inputs.target_rois = target_rois

In [ ]:
pullbetas_workflow = Workflow(name='pullbetas_workflow')
pullbetas_workflow.connect([(infosource2,datagrabber,[('roi','roi'),
                                                      ('list_name','list_name')]),
                            (datagrabber, target_betas,[('betas','sample_betas'),
                                                        ('subjects_list','subject_ids')]),
                            (target_betas, datasink, [('extracted_betas_csv','roi_target_betas')])
                           ])
pullbetas_workflow.base_dir = workflow_dir
#pullbetas_workflow.write_graph(graph2use='flat')
pullbetas_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})